In [28]:
import pandas as pd
from imblearn.over_sampling import SMOTE

### Read and Overview

In [29]:
data = pd.read_csv('data/creditcard.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [30]:
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [31]:
data.shape

(284807, 31)

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

### Check dup

In [33]:
data.drop_duplicates(ignore_index=True, inplace=True)

In [34]:
data.duplicated().any()

np.False_

### Scaler

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
stds = StandardScaler()

In [37]:
data.Time = stds.fit_transform(data[['Time']])
data.Amount = stds.fit_transform(data[['Amount']])

### Build model

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [40]:
X = data.drop(columns=['Class'], axis=1)
y = data['Class']

In [42]:
X.shape, y.shape

((283726, 30), (283726,))

#### Case 1

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [44]:
model_1 = LogisticRegression()

In [45]:
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)
accuracy_score(y_test, y_pred)

0.9991682174476963

In [46]:
confusion_matrix(y_test, y_pred)

array([[70801,    13],
       [   46,    72]])

In [48]:
print(classification_report(y_test, y_pred))
# very low in predicting class 1

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     70814
           1       0.85      0.61      0.71       118

    accuracy                           1.00     70932
   macro avg       0.92      0.80      0.85     70932
weighted avg       1.00      1.00      1.00     70932



#### Case 2

In [51]:
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [52]:
X = data.drop(columns=['Class'], axis=1)
y = data['Class']

In [53]:
X_resample, y_resample = SMOTE().fit_resample(X, y)

In [56]:
sorted(Counter(y_resample).items())

[(0, 283253), (1, 283253)]

In [55]:
model_2 = LogisticRegression()

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample)

In [58]:
model_2.fit(X_train, y_train)
y_pred = model_2.predict(X_test)
accuracy_score(y_test, y_pred)

0.9470863606515707

In [59]:
confusion_matrix(y_test, y_pred)

array([[68816,  1783],
       [ 5711, 65317]])

In [61]:
print(classification_report(y_test, y_pred))
# very high in both precision and accuracy

              precision    recall  f1-score   support

           0       0.92      0.97      0.95     70599
           1       0.97      0.92      0.95     71028

    accuracy                           0.95    141627
   macro avg       0.95      0.95      0.95    141627
weighted avg       0.95      0.95      0.95    141627

